In [1]:
import os
import sys
import pandas as pd
from datetime import datetime

from google.colab import drive

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
ROOT_DIR = '/content/gdrive/MyDrive/Colab Notebooks'
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'disaster_tweets')

In [4]:
print(ROOT_DIR)
print(DATA_DIR)

/content/gdrive/MyDrive/Colab Notebooks
/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets


In [5]:
if ROOT_DIR not in sys.path:
  sys.path.append(ROOT_DIR)

In [6]:
!pip install -r "$ROOT_DIR/requirementscl.txt"

In [7]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for sequence classification."""
import logging
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import datasets
import numpy as np
import tensorflow as tf

from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)
from transformers.utils import logging as hf_logging


hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()


def get_tfds(
    train_file: str,
    eval_file: str,
    test_file: str,
    tokenizer: PreTrainedTokenizer,
    label_column_id: int,
    max_seq_length: Optional[int] = None,
):
    files = {}

    if train_file is not None:
        files[datasets.Split.TRAIN] = [train_file]
    if eval_file is not None:
        files[datasets.Split.VALIDATION] = [eval_file]
    if test_file is not None:
        files[datasets.Split.TEST] = [test_file]

    ds = datasets.load_dataset("csv", data_files=files)
    features_name = list(ds[list(files.keys())[0]].features.keys())
    print(f"Features Name: {features_name}")
    
    label_name = features_name.pop(label_column_id)
    label_list = list(set(ds[list(files.keys())[0]][label_name]))
    label2id = {label: i for i, label in enumerate(label_list)}
    input_names = tokenizer.model_input_names
    transformed_ds = {}

    if len(features_name) == 1:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    example[features_name[0]], truncation=True, max_length=max_seq_length, padding="max_length"
                ),
                batched=True,
            )
    elif len(features_name) == 2:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    (example[features_name[0]], example[features_name[1]]),
                    truncation=True,
                    max_length=max_seq_length,
                    padding="max_length",
                ),
                batched=True,
            )

    def gen_train():
        for ex in transformed_ds[datasets.Split.TRAIN]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_val():
        for ex in transformed_ds[datasets.Split.VALIDATION]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_test():
        for ex in transformed_ds[datasets.Split.TEST]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    train_ds = (
        tf.data.Dataset.from_generator(
            gen_train,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TRAIN in transformed_ds
        else None
    )

    if train_ds is not None:
        train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TRAIN])))

    val_ds = (
        tf.data.Dataset.from_generator(
            gen_val,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.VALIDATION in transformed_ds
        else None
    )

    if val_ds is not None:
        val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.VALIDATION])))

    test_ds = (
        tf.data.Dataset.from_generator(
            gen_test,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TEST in transformed_ds
        else None
    )

    if test_ds is not None:
        test_ds = test_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TEST])))

    return train_ds, val_ds, test_ds, label2id


logger = logging.getLogger(__name__)


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    label_column_id: int = field(metadata={"help": "Which column contains the label"})
    train_file: str = field(default=None, metadata={"help": "The path of the training file"})
    dev_file: Optional[str] = field(default=None, metadata={"help": "The path of the development file"})
    test_file: Optional[str] = field(default=None, metadata={"help": "The path of the test file"})
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )

In [8]:
def run_training(model_name, results_name):
  # See all possible arguments in src/transformers/training_args.py
  # or by passing the --help flag to this script.
  # We now keep distinct sets of args, for a cleaner separation of concerns.
  parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
  model_args, data_args, training_args = parser.parse_args_into_dataclasses()

  if (
      os.path.exists(training_args.output_dir)
      and os.listdir(training_args.output_dir)
      and training_args.do_train
      and not training_args.overwrite_output_dir
  ):
      raise ValueError(
          f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
      )

  # Setup logging
  logging.basicConfig(
      format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
      datefmt="%m/%d/%Y %H:%M:%S",
      level=logging.INFO,
  )
  logger.info(
      "n_replicas: %s, distributed training: %s, 16-bits training: %s",
      training_args.n_replicas,
      bool(training_args.n_replicas > 1),
      training_args.fp16,
  )
  logger.info("Training/evaluation parameters %s", training_args)

  # Load pretrained model and tokenizer
  #
  # Distributed training:
  # The .from_pretrained methods guarantee that only one local process can concurrently
  # download model & vocab.

  tokenizer = AutoTokenizer.from_pretrained(
      model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
      cache_dir=model_args.cache_dir,
  )

  train_dataset, eval_dataset, test_ds, label2id = get_tfds(
      train_file=data_args.train_file,
      eval_file=data_args.dev_file,
      test_file=data_args.test_file,
      tokenizer=tokenizer,
      label_column_id=data_args.label_column_id,
      max_seq_length=data_args.max_seq_length,
  )

  config = AutoConfig.from_pretrained(
      model_args.config_name if model_args.config_name else model_args.model_name_or_path,
      num_labels=len(label2id),
      label2id=label2id,
      id2label={id: label for label, id in label2id.items()},
      finetuning_task="text-classification",
      cache_dir=model_args.cache_dir,
  )

  with training_args.strategy.scope():
      model = TFAutoModelForSequenceClassification.from_pretrained(
          model_args.model_name_or_path,
          from_pt=bool(".bin" in model_args.model_name_or_path),
          config=config,
          cache_dir=model_args.cache_dir,
      )

  def compute_metrics(p: EvalPrediction) -> Dict:
      preds = np.argmax(p.predictions, axis=1)

      return {"acc": (preds == p.label_ids).mean()}

  # Initialize our Trainer
  trainer = TFTrainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
  )

  print(f"train: {len(train_dataset)}")
  print(f"valid: {len(eval_dataset)}")
  print(f"test: {len(test_ds)}")
  print(f"Model name: {model_name}")
  print(f"Results name: {model_results_name}")

  # Training
  if training_args.do_train:
      trainer.train()
      trainer.save_model()
      tokenizer.save_pretrained(training_args.output_dir)

  # Evaluation
  results = {}
  if training_args.do_eval:
      logger.info("*** Evaluate ***")
      result = trainer.evaluate()
      output_eval_file = os.path.join(training_args.output_dir, f"eval_results_{model_results_name}.txt")

      with open(output_eval_file, "w") as writer:
          logger.info("***** Eval results *****")

          for key, value in result.items():
              logger.info("  %s = %s", key, value)
              writer.write("%s = %s\n" % (key, value))

          results.update(result)

          accuracy = results[[k for k in results.keys() if "acc" in k][0]]
          accuracy_str = f"a0{(int(round(accuracy*1000,0)))}"
          print(accuracy_str)
  else:
    acuracy_str = 'aunk'

  # Prediction
  if training_args.do_predict:
      logger.info("*** predictions ***")
      preds = trainer.predict(test_ds)

      logger.info("*** RESUTS: ***")
      logger.info(preds)
      logger.info("*** :RESUTS ***")

      decoded_preds = np.argmax(preds.predictions, axis=1)
      org_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
      org_test['target'] = decoded_preds
      org_test[['id', 'target']].to_csv(os.path.join(DATA_DIR, f'./sub_{accuracy_str}_{model_results_name}.csv'), index=False)


In [ ]:
#############
# MAIN LOOP #
#############
import shutil

# AVAILABLE MODELS:
# ConvBertConfig, DistilBertConfig, AlbertConfig, CamembertConfig, XLMRobertaConfig, LongformerConfig, 
# RobertaConfig, BertConfig, XLNetConfig, MobileBertConfig, FlaubertConfig, XLMConfig, ElectraConfig, FunnelConfig, GPT2Config, MPNetConfig, 
# OpenAIGPTConfig, TransfoXLConfig, CTRLConfig.

models = ['funnel-transformer/medium', 'distilbert-base-uncased', 'roberta-base']

for model_name in models:
  num_epochs = "2"
  timestamp=str(datetime.now()).replace(' ','_').replace(':','').replace('-','').split('.')[0][2:-2]

  model_results_name = f"{model_name}_{num_epochs}e_{timestamp}"
  model_full_path = os.path.join(DATA_DIR, f"mod_{model_name}")

  args = ['run_tf_text_classification.py']
  args.extend([
    "--train_file", os.path.join(DATA_DIR, "train_formatted.csv"),
    "--test_file", os.path.join(DATA_DIR, "test_formatted.csv"),
    "--dev_file", os.path.join(DATA_DIR, "valid_formatted.csv"),
    "--label_column_id", "0",
    "--model_name_or_path", model_name, 
    "--output_dir", model_full_path, 
    "--num_train_epochs", num_epochs,
    "--per_device_train_batch_size", "16",
    "--per_device_eval_batch_size", "32",
    "--do_train", 
    "--do_eval",
    "--do_predict", 
    "--logging_steps", "460", ### 460 for test 256, 444 for test 512, 476 for all records in training
    "--evaluation_strategy", "steps", 
    "--save_steps", "460", ### WAS: "476", ### 2 times per epoch (476 steps per epoch)
    "--overwrite_output_dir", 
    "--max_seq_length", "128"    
  ])

  print(f"*******************")
  print(f"*** Processing: ***")
  print(f"*******************")
  print(f"MODEL NAME: {model_name}")
  print(f"RESULTS NAME: {model_results_name}")
  print(f"TIMESTAMP: {timestamp}")
  
  sys.argv = args

  # Ensure all new files are accessible (it is not always the case for Google Drive)  
  drive.mount('/content/gdrive', force_remount=True)

  run_training(model_name, model_results_name)

  # Clean training artefacts
  logger.info(f"removing checkpoints...")
  shutil.rmtree(os.path.join(model_full_path, 'checkpoint'))
  
  files = os.listdir(model_full_path)
  for f in files:
    if os.path.splitext(f)[1] == '.h5':
      logger.info(f"removing file {f}")
      os.remove(os.path.join(model_full_path, f))

*******************
*** Processing: ***
*******************
MODEL NAME: funnel-transformer/medium
RESULTS NAME: funnel-transformer/medium_2e_210319_1845
TIMESTAMP: 210319_1845


[INFO|training_args.py:631] 2021-03-19 18:45:13,028 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-03-19 18:45:13,104 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-03-19 18:45:13,109 >> Tensorflow: setting up strategy
03/19/2021 18:45:16 - INFO - __main__ -   n_replicas: 1, distributed training: False, 16-bits training: False


Mounted at /content/gdrive


03/19/2021 18:45:16 - INFO - __main__ -   Training/evaluation parameters TFTrainingArguments(output_dir='/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_funnel-transformer/medium', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=32, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, logging_dir='runs/Mar19_18-45-13_a9dc039fb8be', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=460, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=460, save_total_limit=

[INFO|file_utils.py:1390] 2021-03-19 18:45:16,997 >> storing https://huggingface.co/funnel-transformer/medium/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/6917c1e6c6afe2461e94613481d529e038d2937e667f699fd4b25d8b44f73929.667b1a64d5ed1a168f5d9029ec022e41d875ceb44b232fd992af6241c68e0d95
[INFO|file_utils.py:1393] 2021-03-19 18:45:16,998 >> creating metadata file for /root/.cache/huggingface/transformers/6917c1e6c6afe2461e94613481d529e038d2937e667f699fd4b25d8b44f73929.667b1a64d5ed1a168f5d9029ec022e41d875ceb44b232fd992af6241c68e0d95
03/19/2021 18:45:17 - INFO - filelock -   Lock 140097457004816 released on /root/.cache/huggingface/transformers/6917c1e6c6afe2461e94613481d529e038d2937e667f699fd4b25d8b44f73929.667b1a64d5ed1a168f5d9029ec022e41d875ceb44b232fd992af6241c68e0d95.lock
[INFO|configuration_utils.py:463] 2021-03-19 18:45:17,009 >> loading configuration file https://huggingface.co/funnel-transformer/medium/resolve/main/config.json from cache at /root/.cache/

...

[INFO|file_utils.py:1390] 2021-03-19 18:45:31,762 >> storing https://huggingface.co/funnel-transformer/medium/resolve/main/tf_model.h5 in cache at /root/.cache/huggingface/transformers/497e5d335f3530c0c9fa3ead971d0d482a0f683e451a2c2529012e31c7fb808e.eb40de5ff052d00d9f96134e6eca3874aef736f724c63664259a135e311c60fb.h5
[INFO|file_utils.py:1393] 2021-03-19 18:45:31,763 >> creating metadata file for /root/.cache/huggingface/transformers/497e5d335f3530c0c9fa3ead971d0d482a0f683e451a2c2529012e31c7fb808e.eb40de5ff052d00d9f96134e6eca3874aef736f724c63664259a135e311c60fb.h5
03/19/2021 18:45:31 - INFO - filelock -   Lock 140093419898768 released on /root/.cache/huggingface/transformers/497e5d335f3530c0c9fa3ead971d0d482a0f683e451a2c2529012e31c7fb808e.eb40de5ff052d00d9f96134e6eca3874aef736f724c63664259a135e311c60fb.h5.lock
[INFO|modeling_tf_utils.py:1240] 2021-03-19 18:45:31,773 >> loading weights file https://huggingface.co/funnel-transformer/medium/resolve/main/tf_model.h5 from cache at /root/.cach

[WARNING|modeling_tf_utils.py:1290] 2021-03-19 18:46:04,213 >> Some layers from the model checkpoint at funnel-transformer/medium were not used when initializing TFFunnelForSequenceClassification: ['funnel/decoder/layers_._0/attention/post_proj/kernel:0', 'funnel/decoder/layers_._0/ffn/linear_1/kernel:0', 'funnel/decoder/layers_._1/ffn/linear_2/bias:0', 'funnel/decoder/layers_._0/attention/r_r_bias:0', 'funnel/decoder/layers_._0/attention/q_head/kernel:0', 'funnel/decoder/layers_._1/ffn/layer_norm/beta:0', 'funnel/decoder/layers_._1/attention/r_kernel:0', 'funnel/decoder/layers_._1/attention/v_head/kernel:0', 'funnel/decoder/layers_._0/attention/seg_embed:0', 'funnel/decoder/layers_._0/attention/k_head/kernel:0', 'funnel/decoder/layers_._0/ffn/linear_2/bias:0', 'funnel/decoder/layers_._1/attention/r_r_bias:0', 'funnel/decoder/layers_._0/attention/layer_norm/gamma:0', 'funnel/decoder/layers_._1/ffn/linear_1/kernel:0', 'funnel/decoder/layers_._1/attention/v_head/bias:0', 'funnel/decoder/

train: 7357
valid: 256
test: 3263
Model name: funnel-transformer/medium
Results name: funnel-transformer/medium_2e_210319_1845
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


03/19/2021 18:46:11 - WARNING - tensorflow -   AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f6afb28d050>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
